# Freezing Keras Model #

In [1]:
# import keras
import keras
import os
import numpy as np
import time
# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


### Load Model Retinanet ###

In [2]:
model_path='resnet50_coco_best_v2.1.0.h5'

In [3]:
from keras_retinanet import models
model = models.load_model(model_path, backbone_name='resnet50')

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.5/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
originalInputNode = model.input
originalOutputNode = model.output
print("Input node    Name: {} \n{}Shape:{}".format(originalInputNode.name,14*" ",originalInputNode.shape))
for node in originalOutputNode:
    print("Output node   Name: {} \n{}Shape:{}".format(node.name,14*" ",node.shape))

Input node    Name: input_1:0 
              Shape:(?, ?, ?, 3)
Output node   Name: filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0 
              Shape:(?, 300, 4)
Output node   Name: filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0 
              Shape:(?, 300)
Output node   Name: filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3:0 
              Shape:(?, 300)


### Set Static Input Shapes ###
### Rename Input ###
### Rename Output ###

In [5]:
newOutputName=['detection_boxes','detection_scores','detection_classes']
newInputName='image_tensor'
newInputShape=(1,640,640,3)
outPutDir= "some_directory"

In [6]:
from keras.layers import Input
from keras.models import Model
from keras import backend as K

In [7]:
if  newInputName!=None and newInputShape!=None:
    new_input=Input(batch_shape=newInputShape,name=newInputName)
    new_outputs = model(new_input)
    new_model = Model(new_input,new_outputs)
else:
    new_model=Model

In [8]:
print("Input node    Name: {} \n{}Shape:{}".format(new_model.input.name,14*" ",new_model.input.shape))
for node in new_model.output:
    print("Output node   Name: {} \n{}Shape:{}".format(node.name,14*" ",node.shape))

Input node    Name: image_tensor:0 
              Shape:(1, 640, 640, 3)
Output node   Name: retinanet-bbox/filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0 
              Shape:(1, 300, 4)
Output node   Name: retinanet-bbox/filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0 
              Shape:(1, 300)
Output node   Name: retinanet-bbox/filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3:0 
              Shape:(1, 300)


In [9]:
if newOutputName is not None:
    num_output = len(new_model.output)
    for i in range(num_output):
        tf.identity(new_model.outputs[i],name=newOutputName[i])
else:
    newOutputName=[node.name for node in new_model.outputs]
with K.get_session() as sess:
    constant_graph = tf.graph_util.convert_variables_to_constants(
            sess,
            sess.graph.as_graph_def(),
            newOutputName)
    tf.train.write_graph(constant_graph,outPutDir,"frozen.pb", as_text=False)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 306 variables.
INFO:tensorflow:Converted 306 variables to const ops.


# TF-TRT Model #

In [1]:
output_names=['detection_boxes','detection_scores','detection_classes']
frozen_model_path = './some_directory/frozen.pb'
precision = 'FP32'
output_dir='./some_directory'

In [11]:
import tensorflow.contrib.tensorrt as trt
import tensorflow as tf
import numpy as np
import time
def load_pb(pb_path):
    """Load the TF graph from the pre-build pb file."""
    print('------------- Load the TF graph from the pre-build pb file: {} -------------'.format(pb_path))
    start_time = time.time()
    graph_def = tf.GraphDef()
    with tf.gfile.GFile(pb_path, 'rb') as pf:
        graph_def.ParseFromString(pf.read())
    
    stop_time = time.time()
    print('------------- Load time: {0:.2f} sec'.format(stop_time - start_time))
    return graph_def

In [12]:
frozen_graph=load_pb(frozen_model_path)

------------- Load the TF graph from the pre-build pb file: ./some_directory/Frozen.pb -------------
------------- Load time: 0.27 sec


In [16]:
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,
    outputs=output_names,
    max_batch_size=1,
    max_workspace_size_bytes=1<<32,
    precision_mode=precision,
    minimum_segment_size=200
)

INFO:tensorflow:Running against TensorRT version 5.0.2


In [2]:
#!rm './data/Build_trt/fasrcnn_res101/fp16.pb'
with open('{}/{}.pb'.format(output_dir,precision), 'wb') as f:
    f.write(trt_graph.SerializeToString())


FP16.pb  FP32.pb  Frozen.pb


In [3]:
!ls $output_dir

FP16.pb  FP32.pb  Frozen.pb


# TF-Serving Model #

In [19]:
graph_pb = './some_directory/FP16.pb'
export_dir = './tf_serving_model/retinanet_fp16'
map_input = {"in":"image_tensor:0"}
map_output={"boxes":"detection_boxes:0","score":"detection_scores:0","classes":"detection_classes:0"}

In [27]:
import tensorflow as tf
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants
import tensorflow.contrib.tensorrt as trt

builder = tf.saved_model.builder.SavedModelBuilder(export_dir)

with tf.gfile.GFile(graph_pb, "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

sigs = {}

with tf.Session(graph=tf.Graph()) as sess:
    # name="" is important to ensure we don't get spurious prefixing
    tf.import_graph_def(graph_def, name="")
    g = tf.get_default_graph()
#     inp = g.get_tensor_by_name("image_tensor:0")
#     boxes = g.get_tensor_by_name("detection_boxes:0")
#     num_box = g.get_tensor_by_name("num_detections:0")
#     score = g.get_tensor_by_name("detection_scores:0")
#     classes = g.get_tensor_by_name("detection_classes:0")
    map_input_1 ={key: g.get_tensor_by_name(value) for (key, value) in map_input.items()}
    map_output_1 ={key: g.get_tensor_by_name(value) for (key, value) in map_output.items()}
    sigs[signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY] = \
        tf.saved_model.signature_def_utils.predict_signature_def(
            map_input_1,map_output_1)

    builder.add_meta_graph_and_variables(sess,
                                         [tag_constants.SERVING],
                                         signature_def_map=sigs)

builder.save()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./tf_serving_model/retinanet_fp16/saved_model.pb


b'./tf_serving_model/retinanet_fp16/saved_model.pb'